In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

from src.spotify_data import spotify_logs

setup spotipy

In [ ]:
CLIENT_ID = 'CLIENT_ID'
CLIENT_SECRET = 'CLIENT_SECRET'

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

get song history

In [ ]:
songs_played = spotify_logs.read_endsong()

get spotify uris

In [ ]:
min_plays = 0
songs_unique = songs_played[['spotify_track_uri', 'ts']].groupby(by='spotify_track_uri').count()
songs_unique = songs_unique[songs_unique.ts >= min_plays].sort_values('spotify_track_uri')
songs_unique = list(songs_unique.index)
f'{len(songs_unique)} tracks to retrieve'

query and save every 100 tracks

In [ ]:
track_features = pd.DataFrame()

step = 100
for i in range(0, len(songs_unique), step):
    uris = songs_unique[i : i + step]
    features = spotify.audio_features(uris)

    try:
        df_features = pd.DataFrame(features)
    except AttributeError:
        # sometimes the feature is null
        df_features = pd.DataFrame([f for f in features if f])

    track_features = pd.concat([track_features, df_features])
    print(i)

# set index and save
track_features.set_index('uri').to_csv('..\data\\track_features.csv')

In [ ]:
track_features